## Section 1: Operator Overview 
* Using the most recent month

In [1]:
import _report_utils
import _section1_utils as section1
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis.sql import to_snakecase
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
name = "Bay Area 511 AC Transit Schedule"
organization_name = "Alameda-Contra Costa Transit District"

In [5]:
selected_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = selected_date.year
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [6]:
import yaml

# Readable Dictionary
with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

### Map

In [7]:
ac_transit_map = section1.load_operator_map(name)

In [8]:
ac_transit_map.shape

(102, 21)

In [9]:
def plot_route(route):
    filtered_gdf = gdf[gdf["Route"] == route]
    display(
        filtered_gdf.explore(
            column="Route",
            cmap="Spectral",
            tiles="CartoDB positron",
            width=500,
            height=300,
            style_kwds={"weight": 3},
            legend=False,
            tooltip=["Route", "Service Miles"],
        )
    )

### Operator Profiles

In [10]:
ac_transit_profile = section1.load_operator_profiles(organization_name)

In [11]:
ac_transit_profile

,schedule_gtfs_dataset_key,# Routes,# Trips,# Shapes,# Stops,# Arrivals,Operator Service Miles,Avg Arrivals per Stop,# Downtown Local Route Types,# Local Route Types,# Coverage Route Types,# Rapid Route Types,# Express Route Types,# Rail Route Types,Transit Operator,Organization ID,Organization,Date
13,c499f905e33929a641f083dad55c521e,132,5478,346,4733,234582,1459.66,49.56,179,21,44,94,2,0,Bay Area 511 AC Transit Schedule,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,2024-04-17


In [12]:
ac_transit_profile.columns

Index(['schedule_gtfs_dataset_key', '# Routes', '# Trips', '# Shapes',
       '# Stops', '# Arrivals', 'Operator Service Miles',
       'Avg Arrivals per Stop', '# Downtown Local Route Types',
       '# Local Route Types', '# Coverage Route Types', '# Rapid Route Types',
       '# Express Route Types', '# Rail Route Types', 'Transit Operator',
       'Organization ID', 'Organization', 'Date'],
      dtype='object')

#### Original

In [13]:
op_profiles_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"

op_profiles_df = pd.read_parquet(op_profiles_url)

In [14]:
op_profiles_df1 = op_profiles_df.sort_values(by=["service_date"], ascending=False)

In [15]:
op_profiles_df1.columns

Index(['schedule_gtfs_dataset_key', 'operator_n_routes', 'operator_n_trips',
       'operator_n_shapes', 'operator_n_stops', 'operator_n_arrivals',
       'operator_route_length_miles', 'operator_arrivals_per_stop',
       'n_downtown_local_routes', 'n_local_routes', 'n_coverage_routes',
       'n_rapid_routes', 'n_express_routes', 'n_rail_routes', 'name',
       'organization_source_record_id', 'organization_name', 'service_date'],
      dtype='object')

In [16]:
op_profiles_df1.drop_duplicates(subset=["organization_name"])[
    ["service_date"]
].describe()

/tmp/ipykernel_380/3772978131.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  op_profiles_df1.drop_duplicates(subset=["organization_name"])[


,service_date
count,171
unique,11
top,2024-04-17 00:00:00
freq,139
first,2023-03-15 00:00:00
last,2024-04-17 00:00:00


### Operators who don't have profile info from the most recent service_date but have stuff for sched_vp
* City of Torrance	
* Stanislaus Regional Transit Authority

In [17]:
op_profiles2 = op_profiles_df1.drop_duplicates(subset=["organization_name"])[
    ["organization_name", "service_date"]
]

In [18]:
# op_profiles2.loc[op_profiles2.service_date < '2024-04-17'].sort_values(by = ['organization_name'])

### Total Service
* Checking Eric's data.
* Big Blue Bus Schedule
* https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.fct_monthly_route_service_by_timeofday
* TTL_service_hours: Total scheduled service hours that occurred for the route for this month, day_type, and time_of_day.

### Total Monthly Service Hours
Breaking down the `total_monthly_service` function.
1. Load `load_scheduled_service`
2. Count how many Monday's, Tuesday's...Sunday's are in each month-year combo. Ex: May 2024 has 4 Monday's. Merge this dataframe into `load_scheduled_service`
3. Aggregate the data by "Month-Year" combo using `summarize_monthly`. `sum` `ttl_service_hours` after grouping.
4. For `Daily Service Hours`: divide the summed up `ttl_service_hours` by how many days of that day type.


In [25]:
og_df = section1.load_scheduled_service("Big Blue Bus Schedule")

In [26]:
og_df.head(2)

,key,name,schedule_source_record_id,route_id_x,route_short_name,route_long_name,time_of_day,month,year,day_type,n_trips,ttl_service_hours,day_name,schedule_gtfs_dataset_key,organization_source_record_id,organization_name,route_id_y,route_combined_name,datetime_date
318238,131c820d577cabbc588e8ae37d93ef18,Big Blue Bus Schedule,recpN1dPaxhZvZQV0,3554,1,Main St & Santa Monica Blvd/UCLA,Evening,2023-04,2023,5,72,47.07,Thursday,dbbe8ee4864a2715a40749605395d584,recJHFDLpGRMIFgnL,City of Santa Monica,3629,1 Main St & Santa Monica Blvd/UCLA,2023-04-01
318239,6ea1bdf6b8a09827b1e0ad8770621b1c,Big Blue Bus Schedule,recpN1dPaxhZvZQV0,3555,2,Wilshire Blvd/UCLA,AM Peak,2023-04,2023,6,56,33.47,Friday,dbbe8ee4864a2715a40749605395d584,recJHFDLpGRMIFgnL,City of Santa Monica,3630,2 Wilshire Blvd/UCLA,2023-04-01


In [19]:
agg_df = section1.total_monthly_service("Big Blue Bus Schedule")

In [20]:
agg_df.head(2)

,Transit Operator,Month,Time of Day,Day of Week,Total Service (hours),n_days,Daily Service Hours
0,Big Blue Bus Schedule,2023-03,AM Peak,Friday,637.57,5,127.51
1,Big Blue Bus Schedule,2023-03,AM Peak,Monday,425.41,4,106.35


In [23]:
section1.single_bar_chart_dropdown(
    agg_df,
    "Day of Week",
    "Total Service (hours)",
    "Time of Day",
    readable_dict["total_scheduled_hours"]["title"],
    "Month",
    readable_dict["total_scheduled_hours"]["subtitle"],
)

alt.Chart(...)

In [24]:
section1.single_bar_chart_dropdown(
    agg_df,
    "Day of Week",
    "Daily Service Hours",
    "Time of Day",
    "Daily Scheduled Service Hours by Time of Day",
    "Month",
    readable_dict["total_scheduled_hours"]["subtitle"],
)

alt.Chart(...)